<a href="https://colab.research.google.com/github/saadAwd/Kaggle/blob/main/HIKPbzdnTyF4eYWh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy

In [2]:
!python -m spacy download en_core_web_md


2023-09-29 12:53:47.042575: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 12:53:48.157801: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
import os
print(os.getcwd())

/content


In [4]:
from google.colab import files
uploaded = files.upload()

Saving validated.tsv to validated.tsv


In [5]:
import pandas as pd



# Define the file paths
original_file_path = "/content/validated.tsv"
new_file_path = "/content/sampled_real_voices.tsv"

# Read the original dataset
df_original = pd.read_csv(original_file_path, delimiter='\t')

# Randomly select 165 rows from the original dataset
df_sampled = df_original.sample(n=165, random_state=42)  # Adjust the random_state if needed

# Save the sampled dataset to a new file
df_sampled.to_csv(new_file_path, sep='\t', index=False)

In [6]:
import csv
import spacy
from itertools import islice

# Load the pre-trained word embeddings model
nlp = spacy.load("en_core_web_md")

# Path to the "other" TSV file containing the MP3 paths and sentences
sampled_file_path = "/content/sampled_real_voices.tsv"

# Batch size for processing sentences
batch_size = 100

# List to store the embedded vectors
embedded_vectors = []

# Read the TSV file in chunks
with open(sampled_file_path, "r") as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')

    for chunk in iter(lambda: list(islice(reader, batch_size)), []):
        mp3_paths = []
        sentences = []

        for row in chunk:
            if len(row) >= 2:
                mp3_paths.append(row[1].strip())
                sentences.append(row[2].strip())

        # Perform text embedding on the sentences
        for sentence in sentences:
            doc = nlp(sentence)

            # Access the embedded vectors
            for token in doc:
                embedded_vectors.append(token.vector)

        # Perform text embedding on the MP3 paths
        for mp3_path in mp3_paths:
            doc = nlp(mp3_path)

            # Access the embedded vectors
            for token in doc:
                embedded_vectors.append(token.vector)

# Save the embedded vectors to a CSV file
output_file = "/content/real_embedded_vectors.npy"  # Update with the desired output file path

with open(output_file, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(embedded_vectors)

In [7]:
from google.colab import files
uploaded = files.upload()

Saving cloned_df.csv to cloned_df.csv


In [15]:
import pandas as pd
import spacy
import numpy as np

# Load the pre-trained word embeddings model
nlp = spacy.load("en_core_web_md")

# Path to the CSV file containing the fake voice data
fake_voices_file_path = "/content/cloned_df.csv"

# Batch size for processing sentences
batch_size = 100

# List to store the embedded vectors
embedded_vectors = []

# Read the CSV file in chunks
for chunk in pd.read_csv(fake_voices_file_path, chunksize=batch_size):
    # Extract the sentences from the chunk
    sentences = chunk["text"].tolist()

    # Perform text embedding on the sentences
    for sentence in sentences:
        # Perform text embedding on the sentence
        doc = nlp(sentence)

        # Access the embedded vectors
        for token in doc:
            embedded_vectors.append(token.vector)

# Convert the list of embedded vectors to a NumPy array
embedded_vectors = np.array(embedded_vectors)

# Save the embedded vectors to a file
output_file_path = "/content/fake_embedded_vectors.csv"  # Update with the desired output file path
np.save(output_file_path, embedded_vectors)

In [16]:
def extract_acoustic_features(audio_file):
    # Load the audio file
    audio, sr = librosa.load(audio_file)

    # Extract acoustic features
    features = {}
    features['duration'] = len(audio) / sr
    features['tempo'] = librosa.beat.tempo(y=audio, sr=sr)[0]
    features['spectral_centroid'] = librosa.feature.spectral_centroid(y=audio, sr=sr)[0].mean()

    # Add more features as needed

    return features

In [18]:
from google.colab import files
uploaded = files.upload()

Saving real_embedded_vectors.csv to real_embedded_vectors.csv


In [17]:
import os
import pandas as pd
import librosa
import csv

directory = '/content/cv-corpus-150-new'
tsv_file_path = '/content/sampled_real_voices.tsv'

# Open the TSV file
with open(tsv_file_path, 'r') as tsvfile:
    # Read the TSV file using csv.reader
    reader = csv.reader(tsvfile, delimiter='\t')

    # Skip the header row if present
    next(reader)

    # Create empty lists to store features and labels
    features_list = []
    labels_list = []

    # Iterate over each row in the TSV file
    for row in reader:
        audio_file_name = row[1]  # Assuming the audio file path is in the first column
        label = row[0]  # Assuming the label is in the second column

        # Search for the audio file recursively within the directory
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file == audio_file_name:
                    audio_file_path = os.path.join(root, file)

                    # Process the audio file and extract acoustic features
                    print(f"Processing audio file: {audio_file_path}")
                    # Example: Extract acoustic features using a function called "extract_acoustic_features"
                    features = extract_acoustic_features(audio_file_path)
                    # Append the features and label to the respective lists
                    features_list.append(features)
                    labels_list.append(label)
                    break  # Exit the inner loop once the file is found
            else:
                continue  # Continue searching in the next subdirectory
            break  # Exit the outer loop once the file is found
        else:
            print(f"Audio file not found: {audio_file_name}")

    # Create a pandas DataFrame from the features and labels
    df_features = pd.DataFrame(features_list)
    df_labels = pd.DataFrame(labels_list, columns=['label'])
    df_combined = pd.concat([df_features, df_labels], axis=1)

    # Save the extracted features and labels to a CSV file
    output_csv_file = '/content/real_features.csv'  # Update with the desired output file path
    df_combined.to_csv(output_csv_file, index=False)

In [11]:
from google.colab import files
uploaded = files.upload()

Saving real_features.csv to real_features.csv


In [20]:
from google.colab import files
uploaded = files.upload()

Saving fake_embedded_vectors.npy to fake_embedded_vectors.npy


In [ ]:
import os
import pandas as pd
import librosa
import csv
import numpy as np

directory = '/content/colab'
csv_file_path = '/content/cloned-voices/cloned_df.csv'

# Define the function to extract acoustic features from the audio file
def extract_acoustic_features(audio_file_path):
    # Load the audio file
    audio, sr = librosa.load(audio_file_path, sr=None)

    # Extract MFCC features
    features = librosa.feature.mfcc(y=audio, sr=sr)

    return features.flatten()  # Flatten the feature array

# Open the CSV file
with open(csv_file_path, 'r') as csvfile:
    # Read the CSV file using csv.reader
    reader = csv.reader(csvfile)

    # Skip the header row if present
    next(reader)

    # Create empty lists to store features and labels
    features_list = []
    labels_list = []

    # Iterate over each row in the CSV file
    for row in reader:
        audio_file_name = row[0]  # Assuming the audio file path is in the second column
        label = row[1]  # Assuming the label is in the first column

        # Search for the audio file recursively within the directory
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file == audio_file_name:
                    audio_file_path = os.path.join(root, file)

                    # Process the audio file and extract acoustic features
                    print(f"Processing audio file: {audio_file_path}")
                    features = extract_acoustic_features(audio_file_path)

                    # Append the features and label to the respective lists
                    features_list.append(features)
                    labels_list.append(label)

                    break  # Exit the inner loop once the file is found
            else:
                continue  # Continue searching in the next subdirectory
            break  # Exit the outer loop once the file is found
        else:
            print(f"Audio file not found: {audio_file_name}")

    # Create a pandas DataFrame from the features and labels
    df_features = pd.DataFrame(features_list)
    df_labels = pd.DataFrame(labels_list, columns=['label'])
    df_combined = pd.concat([df_features, df_labels], axis=1)

    # Save the extracted features and labels to a CSV file
    output_csv_file = '/content/fake_features.csv'  # Update with the desired output file path
    df_combined.to_csv(output_csv_file, index=False)

In [12]:
from google.colab import files
uploaded = files.upload()

Saving fake_features (1).csv to fake_features (1).csv


In [13]:
from google.colab import files
uploaded = files.upload()

Saving real_features.csv to real_features (1).csv


In [21]:
import pandas as pd
import numpy as np

# Load real voice text embeddings
real_data = pd.read_csv('/content/real_embedded_vectors.csv')

# Load fake voice text embeddings from .npy file
fake_data = np.load('/content/fake_embedded_vectors.npy')

# Check the number of columns in real and fake data
if real_data.shape[1] != fake_data.shape[1]:
    print("Real data shape:", real_data.shape)
    print("Fake data shape:", fake_data.shape)
    raise ValueError("Number of columns in real and fake data do not match.")

# Duplicate the fake data to match the number of rows in the real data
num_duplicates = real_data.shape[0] // fake_data.shape[0]
fake_data = np.tile(fake_data, (num_duplicates, 1))

# Truncate the duplicated fake data to match the number of rows in the real data
num_extra_rows = real_data.shape[0] % fake_data.shape[0]
fake_data = np.concatenate((fake_data, fake_data[:num_extra_rows]), axis=0)

# Create a DataFrame for fake voice embeddings
fake_data = pd.DataFrame(fake_data, columns=real_data.columns)

# Add a column 'label' to indicate the class (0 for real, 1 for fake)
real_data['label'] = 0
fake_data['label'] = 1

# Concatenate the real and fake data
data = pd.concat([real_data, fake_data], ignore_index=True)

# Separate the features (text embeddings) and the target variable
X = data.drop('label', axis=1)
y = data['label']



In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a logistic regression classifier with increased max_iter
logreg = LogisticRegression(max_iter=1000)  # Set max_iter to your desired value
logreg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logreg.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 66.11%


In [23]:
from google.colab import files
uploaded = files.upload()

Saving fake voice.wav to fake voice.wav


In [24]:
!pip install speechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 29.4 MB/s eta 0:00:00


In [26]:
import pandas as pd
import numpy as np
import spacy
import joblib
from scipy.io import wavfile
import speech_recognition as sr

# Load the Spacy model with pre-trained word embeddings
nlp = spacy.load("en_core_web_md")


def speech_to_text(audio):
    """
    Return the text from the audio using automatic speech recognition (ASR).
    """
    r = sr.Recognizer()
    with sr.AudioFile(audio) as source:
        audio_data = r.record(source)
        text = r.recognize_google(audio_data)
        return text


# Load the voice file paths
voice_file_paths = ['/content/fake voice.wav']

# List to store the transcribed texts
transcribed_texts = []

# Process the voice files
for voice_file in voice_file_paths:
    # Convert voice data to text using ASR
    transcribed_text = speech_to_text(voice_file)

    # Append the transcribed text to the list
    transcribed_texts.append(transcribed_text)

# Create a DataFrame for the transcribed texts
transcribed_texts_df = pd.DataFrame({'Transcribed Text': transcribed_texts})

# Now you can use the transcribed texts for further processing or analysis
# For example, you can pass the transcribed texts to the Spacy model to extract embeddings
voice_file_embeddings = []

for transcribed_text in transcribed_texts:
    # Generate the text embedding using Spacy
    doc = nlp(transcribed_text)

    # Calculate the average of token vectors for the text embedding
    text_embedding = np.mean([token.vector for token in doc], axis=0)

    voice_file_embeddings.append(text_embedding)

# Create a DataFrame for the voice file embeddings
voice_file_embeddings = pd.DataFrame(voice_file_embeddings)


# Use the trained logistic regression classifier for prediction
predictions = logreg.predict(voice_file_embeddings)

if predictions == 1:
    print("This is a Fake voice")
else:
    print("This is a real voice")

This is a Fake voice


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [27]:
from google.colab import files
uploaded = files.upload()

Saving real voice.mp3 to real voice.mp3


In [30]:
import pandas as pd
import numpy as np
import spacy
import joblib
from scipy.io import wavfile
import speech_recognition as sr
import subprocess

def convert_audio(input_file, output_file):
    subprocess.run(['ffmpeg', '-i', input_file, output_file])

# Provide the path to the input MP3 file and the desired output WAV file
input_file = '/content/real voice.mp3'
output_file = '/content/test_real_file.wav'

# Convert the MP3 file to WAV format
convert_audio(input_file, output_file)
# Load the Spacy model with pre-trained word embeddings
nlp = spacy.load("en_core_web_md")


def speech_to_text(audio):
    """
    Return the text from the audio using automatic speech recognition (ASR).
    """
    r = sr.Recognizer()
    with sr.AudioFile(audio) as source:
        audio_data = r.record(source)
        text = r.recognize_google(audio_data)
        return text


# Load the voice file paths
voice_file_paths = ['/content/test_real_file.wav']

# List to store the transcribed texts
transcribed_texts = []

# Process the voice files
for voice_file in voice_file_paths:
    # Convert voice data to text using ASR
    transcribed_text = speech_to_text(voice_file)

    # Append the transcribed text to the list
    transcribed_texts.append(transcribed_text)

# Create a DataFrame for the transcribed texts
transcribed_texts_df = pd.DataFrame({'Transcribed Text': transcribed_texts})

# Now you can use the transcribed texts for further processing or analysis
# For example, you can pass the transcribed texts to the Spacy model to extract embeddings
voice_file_embeddings = []

for transcribed_text in transcribed_texts:
    # Generate the text embedding using Spacy
    doc = nlp(transcribed_text)

    # Calculate the average of token vectors for the text embedding
    text_embedding = np.mean([token.vector for token in doc], axis=0)

    voice_file_embeddings.append(text_embedding)

# Create a DataFrame for the voice file embeddings
voice_file_embeddings = pd.DataFrame(voice_file_embeddings)


# Use the trained logistic regression classifier for prediction
predictions = logreg.predict(voice_file_embeddings)

if predictions == 1:
    print("This is a Fake voice")
else:
    print("This is a real voice")

This is a real voice


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
